In [1]:
#-*- coding:utf8-*-
%matplotlib inline


from __future__ import absolute_import, division, print_function, unicode_literals

from IPython.display import Image

import numpy as np

import uncertainties as unc
from uncertainties import unumpy as unp
from uncertainties.unumpy import std_devs as stds #TODO
from uncertainties.unumpy import nominal_values as noms #TODO

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D 

import seaborn as sns
sns.set(context="paper",font_scale=1.4)

import scipy as sp
import scipy.constants as const

import sympy as sym
import sympy.abc
from sympy import init_printing


import pandas as pd

import ROOT as rt

import root_numpy as rnp

from aputils.latextables.tables import Table


# Fehlerfortpflanzung beim Mitteln
umean = unc.wrap(np.mean)

#Laden des Latex fonts 
plt.rcParams['text.latex.preamble']=["\\usepackage{lmodern}"]
#Options
params = {'text.usetex' : True,
          'font.size' : 11,
          'font.family' : 'lmodern',
          'text.latex.unicode': True,
          }
plt.rcParams.update(params) 

#Größe der Achsenbeschriftung
axis_font = {'size':'14'}

# Pfade
path_graphics = "../Grafiken/"
path_table = "../Tabellen/"
path_data = "../Messdaten/"

In [3]:
# Funktionen
# Achsenbeschriftungen
def axis_label(label,sym=None,unit=None):
    if sym == None:
        return lable
    elif unit == None:
        return r"".join([label," ${s}$".format(s=sym)])
    else:
        return r"".join([label," ${s}/\mathrm{{{u}}}$".format(s=sym,u=unit)])
    
# Fit Funktionen
def fit_errors(popt,pcov):
    errors = np.sqrt(np.diag(pcov))
    param_err = unp.uarray(popt,errors)
    return param_err
    
def print_errors(array,description=None):
    if description and len(description)==len(array):
        maxlen = max(map(len,description))
        lens = map(len,description)
        for i,entry in enumerate(array):
            print(description[i]," "*(maxlen-lens[i]),entry.nominal_value,"+/-",entry.std_dev)
    else:
        for i,entry in enumerate(array):
            print(entry.nominal_value,"+/-",entry.std_dev)


##Klassen für die Bauteile

In [6]:
class Pipe:
    def __init__(self,diameter=0,length=0,diam_err=0,len_err=0):
        self.diameter = unc.ufloat(diameter,diam_err)
        self.length = unc.ufloat(length,len_err)
        
    @property
    def volume(self):
        return const.pi * (self.diameter**2/4) * self.length
        #self.eq_volume = self._volume_(self.diameter,self.length,eq=True)

class Tank(Pipe):
    def __init__(self,diameter=0,wall_thickness=0,length=0,diam_err=0,wall_err=0,len_err=0):
        self.diameter_outter = unc.ufloat(diameter,diam_err)
        self.wall_thickness = unc.ufloat(wall_thickness,wall_err)
        self.diameter = self.diameter_outter - self.wall_thickness
        self.length = unc.ufloat(length,len_err)
        
class TPipe:
    def __init__(self,diameter=0,length_l=0,length_s=0,diam_err=0,len_l_err=0,len_s_err=0):
        self.diameter = unc.ufloat(diameter,diam_err)
        self.length_long = unc.ufloat(length_l,len_l_err)
        self.length_short = unc.ufloat(length_s,len_s_err)
    @property
    def volume(self):    
        return const.pi * (self.diameter**2/4) * (self.length_long + self.length_short)
    
class XPipe:
    def __init__(self,diameter_l=0,diameter_s=0,length_l=0,length_s=0,
                 diam_l_err=0,diam_s_err=0,len_l_err=0,len_s_err=0):
        self.diameter_long = unc.ufloat(diameter_l,diam_l_err)
        self.diameter_short = unc.ufloat(diameter_s,diam_s_err)
        self.length_long = unc.ufloat(length_l,len_l_err)
        self.length_short = unc.ufloat(length_s,len_s_err)
        
    @property  
    def volume(self):
        return const.pi/4 * ((self.diameter_long**2 * self.length_long) + 
                            2*(self.diameter_short**2 * self.length_short))
class Valve(Pipe):
    def __init__(self,diameter=0,length=0,diam_err=0,len_err=0):
        Pipe.__init__(self,diameter=diameter,length=length,diam_err=diam_err,len_err=len_err)
    @property
    def volume_closed(self): 
        return self.volume/2  

### Verwendete Bauteile

In [20]:
# Tank
tank_front_connection = Pipe(diameter=38.5,length=130,diam_err=0.1,len_err=5) #mm
tank_maintank = Tank(diameter=155,diam_err=5,wall_thickness=4,wall_err=2,length=480,len_err=10)
tank_back_connection = Tank(diameter=28,diam_err=0.1,wall_thickness=2,wall_err=1,length=75,len_err=5)
tank_side_connection = Tank(diameter=38.3,diam_err=0.1,wall_thickness=4,wall_err=2,length=75,len_err=5)

In [21]:
print("Rezipienten Volumen",(tank_front_connection.volume + tank_maintank.volume + 
tank_back_connection.volume + tank_side_connection.volume)/1000000,"l")

Rezipienten Volumen 8.9+/-0.6 l


In [22]:
# Rohre
pipe_recipient_2_xpipe = Pipe(diameter=38,length=400,diam_err=5,len_err=5) #mm
pipe_smallest_0 =  Pipe(diameter=15,length=240,diam_err=2,len_err=5)
pipe_smallest_1 =  Pipe(diameter=15,length=240,diam_err=2,len_err=5)
pipe_xpipe_2_rotarypump = Pipe(diameter=15,length=440,diam_err=2,len_err=5)
flange_turbopump = Pipe(diameter=40,length=52,diam_err=1,len_err=5)

# X- und T-Stücke
xpipe_big = XPipe(diameter_l=39.5,diameter_s=16,length_l=130,length_s=40, # Kurze Längen und Fehler angepasst
                 diam_l_err=0.1,diam_s_err=0.1,len_l_err=5,len_s_err=5)   # Mittelwert der zwei Längen bis zum Zylinder
xpipe_small = XPipe(diameter_l=11,diameter_s=11,length_l=80,length_s=70, 
                 diam_l_err=0.1,diam_s_err=0.1,len_l_err=5,len_s_err=5)
tpipe_big = TPipe(diameter=40,length_l=129,length_s=46,diam_err=1,len_l_err=5,len_s_err=1)
tpipe_small = TPipe(diameter=11,length_l=80,length_s=36,diam_err=0.1,len_l_err=5,len_s_err=5)
tpipe_turbopump = TPipe(diameter=40,length_l=110,length_s=60,diam_err=1,len_l_err=20,len_s_err=10)

#Ventile
valve_lever_0 = Valve(diameter=15,length=77,diam_err=1,len_err=5) # Fehler angepasst
valve_lever_1 = Valve(diameter=15,length=77,diam_err=1,len_err=5)
valve_lever_2 = Valve(diameter=15,length=77,diam_err=1,len_err=5)
valve_lever_3 = Valve(diameter=15,length=77,diam_err=1,len_err=5)
valve_turbopump = Valve(diameter=40,length=33,diam_err=1,len_err=5)

###  Volumen des gesamten Rezipienten für die einzelnen Versuche 

### Laden der Daten